In [1]:
import json
import time
import os
import re

from modules.preprocessing_protocols import preprocess_text
from modules.extract_results import extract_results_to_txt_file, display_results_by_subdistribution,\
                                    display_results_by_named_entity, display_confusion_matrix

from estnltk import Text
from estnltk.taggers import NerTagger
from estnltk.taggers import WordLevelNerTagger
from estnltk.converters import text_to_json
from estnltk.converters import json_to_text
from estnltk.layer_operations import flatten

from estnltk.taggers.estner.ner_trainer import NerTrainer
from estnltk.taggers.estner.model_storage_util import ModelStorageUtil

In [2]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

These files don't work because the protocols are written in a different language, which the goldstandard didn't recognise, hence have no goldstandard tags.

In [3]:
files_not_working = ['J2rva_Tyri_V22tsa_id22177_1911a.json', \
                     'J2rva_Tyri_V22tsa_id18538_1894a.json', \
                     'J2rva_Tyri_V22tsa_id22155_1911a.json', \
                     'Saare_Kihelkonna_Kotlandi_id18845_1865a.json', \
                     'P2rnu_Halliste_Abja_id257_1844a.json', \
                     'Saare_Kaarma_Loona_id7575_1899a.json', \
                     'J2rva_Tyri_V22tsa_id22266_1913a.json', \
                     'J2rva_Tyri_V22tsa_id22178_1912a.json']

models = ['model_morph_with_lemmas_and_sentences', \
          'model_morph_with_lemmas_and_sentences_and_gazzetteer', \
          'model_morph_without_lemmas']

In [4]:
files = {}

with open(os.path.join('..', 'data', 'divided_corpus.txt'), 'r', encoding = 'UTF-8') as f:
    txt = f.readlines()

for fileName in txt:
    file, subdistribution = fileName.split(":")
    files[file] = subdistribution.rstrip("\n")

In [5]:
for model_directory in models:
    for subdistribution in [1, 2, 3, 4, 5]:
        training_subdistributions = []
        for y in [1, 2, 3, 4, 5]:
            if y == subdistribution:
                subdistribution_for_testing = y
            else:
                training_subdistributions.append(y)

            # Getting the filenames to be trained on from the files dictionary.
        filenames = {key: value for key, value in files.items() if int(value) in training_subdistributions}

        # Creating training_texts from the aforementioned filenames.
        print("Valmistan ette treenimistekste.")
        start = time.time()
        training_texts = []
        for filename in filenames:
            with open(os.path.join('..', 'data', 'vallakohtufailid-json-flattened', filename), 'r', encoding='UTF-8') as file:
                if filename in files_not_working:
                    continue
                else:
                    training_texts.append(preprocess_text(json_to_text(file.read())))
        print(f"Treenimistekstid ette valmistatud {time.time() - start} sekundiga.")
        # Setting up the trainer and training.
        print("Alustan NerTaggeri treenimist.")
        start = time.time()
        new_model_dir = os.path.join('models', model_directory)
        modelUtil = ModelStorageUtil( new_model_dir )
        nersettings = modelUtil.load_settings()
        trainer = NerTrainer(nersettings)
        trainer.train( training_texts, layer='gold_wordner', model_dir=new_model_dir )
        print(f"NerTagger treenitud {time.time() - start} sekundiga.")

        # Setting up the new trained nertagger and defining layers to be removed later on.
        nertagger = NerTagger(model_dir = new_model_dir)
        removed_layers = ['sentences', 'morph_analysis', 'compound_tokens', 'ner', 'words', 'tokens']

        # Tagging the files using the new nertagger.
        print("\n\nAlustan failide märgendamist.")
        start = time.time()
        testing_files = {key: value for key, value in files.items() if int(value) == subdistribution_for_testing}
        for file in testing_files:
            with open(find(file.replace(".json", ".txt"), os.path.join('..', 'data', 'vallakohtufailid')), 'r', encoding='UTF-8') as f:
                text = f.read()
                if file == "Tartu_V6nnu_Ahja_id3502_1882a.txt":
                    text = text.replace('..', '. .')
                text = preprocess_text(Text(text))
                nertagger.tag(text)
                text.add_layer(flatten(text['ner'], 'flat_ner'))

                for x in removed_layers:
                    text.pop_layer(x)

                text_to_json(text, file=os.path.join(new_model_dir, 'vallakohtufailid-trained-nertagger', file))
                print(f'Märgendatud fail {file}')
        print(f"Failid märgendatud {time.time() - start} sekundiga.")
    extract_results_to_txt_file(model_directory, files)
print("Programm on lõpetanud oma töö.")

Valmistan ette treenimistekste.
Treenimistekstid ette valmistatud 106.3345000743866 sekundiga.
Alustan NerTaggeri treenimist.
(!) Warning: Location of the new "settings.py" is the same one as the old one. Model's settings are not copied.
Feature generation
type: CRF1d
feature.minfreq: 0.000000
feature.possible_states: 0
feature.possible_transitions: 0
0....1....2....3....4....5....6....7....8....9....10
Number of features: 22088
Seconds required: 0.197

Stochastic Gradient Descent (SGD)
c2: 0.001000
max_iterations: 1000
period: 10
delta: 0.000001

Calibrating the learning rate (eta)
calibration.eta: 0.100000
calibration.rate: 2.000000
calibration.samples: 1000
calibration.candidates: 10
calibration.max_trials: 20
Initial loss: 30999.990087
Trial #1 (eta = 0.100000): 3748.069102
Trial #2 (eta = 0.200000): 5376.620239
Trial #3 (eta = 0.400000): 9877.432780
Trial #4 (eta = 0.800000): 16289.123185
Trial #5 (eta = 1.600000): 34870.702753 (worse)
Trial #6 (eta = 0.050000): 3388.817656
Trial 

***** Epoch #38 *****
Loss: 4491.842570
Improvement ratio: 0.208676
Feature L2-norm: 131.821026
Learning rate (eta): 0.049811
Total number of feature updates: 617234
Seconds required for this iteration: 0.206

***** Epoch #39 *****
Loss: 4554.704595
Improvement ratio: 0.126560
Feature L2-norm: 133.135904
Learning rate (eta): 0.049806
Total number of feature updates: 633477
Seconds required for this iteration: 0.207

***** Epoch #40 *****
Loss: 5104.698906
Improvement ratio: -0.023896
Feature L2-norm: 134.738982
Learning rate (eta): 0.049801
Total number of feature updates: 649720
Seconds required for this iteration: 0.207

SGD terminated with the stopping criteria
Loss: 4491.842570
Total seconds required for training: 8.517

Storing the model
Number of active features: 22088 (22088)
Number of active attributes: 15336 (15336)
Number of active labels: 11 (11)
Writing labels
Writing attributes
Writing feature references for transitions
Writing feature references for attributes
Seconds req

Märgendatud fail V6ru_R2pina_Kahkva_id24938_1868a.json
Märgendatud fail J2rva_Tyri_S2revere_id12762_1879a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id21934_1881a.json
Märgendatud fail V6ru_Vastseliina_Misso_id7468_1885a.json
Märgendatud fail Saare_K2rla_K2rla_id5736_1827a.json
Märgendatud fail Tartu_Sangaste_Kuigatsi_id16414_1872a.json
Märgendatud fail Harju_Kose_Kose-Uuem6isa_id2174_1867a.json
Märgendatud fail Viljandi_P6ltsamaa_Pajusi_id2717_1871a.json
Märgendatud fail Viljandi_P6ltsamaa_Vana-P6ltsamaa_id8104_1888a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id7189_1884a.json
Märgendatud fail Tartu_V6nnu_Kiidj2rve_id25125_1870a.json
Märgendatud fail L22ne_Reigi_K6rgessaare_id22613_1892a.json
Märgendatud fail J2rva_Peetri_V2ike-Kareda_id19114_1867a.json
Märgendatud fail Harju_Hageri_Kohila_id22158_1890a.json
Märgendatud fail P2rnu_P2rnu-Elisabethi_Sauga_id17814_1868a.json
Märgendatud fail Harju_Kose_Palvere_id561_1867a.json
Märgendatud fail V6ru_R2pina_R2pina_id21267_1867a.jso

***** Epoch #1 *****
Loss: 23038.435235
Feature L2-norm: 31.965118
Learning rate (eta): 0.049995
Total number of feature updates: 16201
Seconds required for this iteration: 0.229

***** Epoch #2 *****
Loss: 14842.586147
Feature L2-norm: 41.653617
Learning rate (eta): 0.049990
Total number of feature updates: 32402
Seconds required for this iteration: 0.229

***** Epoch #3 *****
Loss: 13085.328539
Feature L2-norm: 48.883563
Learning rate (eta): 0.049985
Total number of feature updates: 48603
Seconds required for this iteration: 0.231

***** Epoch #4 *****
Loss: 11392.371621
Feature L2-norm: 54.463001
Learning rate (eta): 0.049980
Total number of feature updates: 64804
Seconds required for this iteration: 0.234

***** Epoch #5 *****
Loss: 10472.966677
Feature L2-norm: 59.360345
Learning rate (eta): 0.049975
Total number of feature updates: 81005
Seconds required for this iteration: 0.232

***** Epoch #6 *****
Loss: 10163.444528
Feature L2-norm: 64.039148
Learning rate (eta): 0.049970
Tot

***** Epoch #42 *****
Loss: 4347.055663
Improvement ratio: 0.096402
Feature L2-norm: 137.145225
Learning rate (eta): 0.049791
Total number of feature updates: 680442
Seconds required for this iteration: 0.250

***** Epoch #43 *****
Loss: 4219.788690
Improvement ratio: 0.244700
Feature L2-norm: 138.324548
Learning rate (eta): 0.049786
Total number of feature updates: 696643
Seconds required for this iteration: 0.255

***** Epoch #44 *****
Loss: 4482.441246
Improvement ratio: 0.083346
Feature L2-norm: 139.586200
Learning rate (eta): 0.049781
Total number of feature updates: 712844
Seconds required for this iteration: 0.243

***** Epoch #45 *****
Loss: 4438.691999
Improvement ratio: 0.048236
Feature L2-norm: 140.927070
Learning rate (eta): 0.049776
Total number of feature updates: 729045
Seconds required for this iteration: 0.240

***** Epoch #46 *****
Loss: 4212.206297
Improvement ratio: 0.078866
Feature L2-norm: 142.075491
Learning rate (eta): 0.049771
Total number of feature updates: 7

...

Märgendatud fail L22ne_Vormsi_Vormsi_id14921_1888a.json
Märgendatud fail L22ne_Vormsi_Vormsi_id25012_1888a.json
Märgendatud fail J2rva_Tyri_V22tsa_id19025_1898a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16384_1885a.json
Märgendatud fail Harju_Kose_Palvere_id21108_1885a.json
Märgendatud fail Tartu_R6ngu_Aakre_id4809_1889a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id13022_1876a.json
Märgendatud fail Tartu_R6ngu_Aakre_id8898_1827a.json
Märgendatud fail Tartu_V6nnu_Ahja_id22170_1868a.json
Märgendatud fail Tartu_Otep22_Pyhaj2rve_id22089_1886a.json
Märgendatud fail J2rva_Anna_Purdi_id25497_1870a.json
Märgendatud fail Tartu_V6nnu_Ahja_id17890_1885a.json
Märgendatud fail Viljandi_P6ltsamaa_Pajusi_id6831_1871a.json
Märgendatud fail L22ne_Pyhalepa_K2rdla_id23188_1872a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id1584_1878a.json
Märgendatud fail Viljandi_P6ltsamaa_Uue-P6ltsamaa_id6410_1853a.json
Märgendatud fail L22ne_Martna_Martna_id17524_1871a.json
Märgendatud fail V6ru_Vastseliina_Mis

***** Epoch #7 *****
Loss: 8661.582673
Feature L2-norm: 72.280136
Learning rate (eta): 0.049965
Total number of feature updates: 109298
Seconds required for this iteration: 0.220

***** Epoch #8 *****
Loss: 8203.800364
Feature L2-norm: 75.855171
Learning rate (eta): 0.049960
Total number of feature updates: 124912
Seconds required for this iteration: 0.224

***** Epoch #9 *****
Loss: 7419.842108
Feature L2-norm: 79.206310
Learning rate (eta): 0.049955
Total number of feature updates: 140526
Seconds required for this iteration: 0.229

***** Epoch #10 *****
Loss: 7051.931646
Feature L2-norm: 82.388774
Learning rate (eta): 0.049950
Total number of feature updates: 156140
Seconds required for this iteration: 0.252

***** Epoch #11 *****
Loss: 6690.081193
Improvement ratio: 2.670992
Feature L2-norm: 85.355033
Learning rate (eta): 0.049945
Total number of feature updates: 171754
Seconds required for this iteration: 0.225

***** Epoch #12 *****
Loss: 6615.452972
Improvement ratio: 1.255103
Fe

***** Epoch #49 *****
Loss: 3326.276678
Improvement ratio: 0.105418
Feature L2-norm: 147.103546
Learning rate (eta): 0.049756
Total number of feature updates: 765086
Seconds required for this iteration: 0.229

***** Epoch #50 *****
Loss: 3327.408018
Improvement ratio: 0.092899
Feature L2-norm: 148.134517
Learning rate (eta): 0.049751
Total number of feature updates: 780700
Seconds required for this iteration: 0.242

***** Epoch #51 *****
Loss: 3352.502272
Improvement ratio: 0.067132
Feature L2-norm: 149.104074
Learning rate (eta): 0.049746
Total number of feature updates: 796314
Seconds required for this iteration: 0.224

***** Epoch #52 *****
Loss: 3238.755453
Improvement ratio: 0.150887
Feature L2-norm: 150.067780
Learning rate (eta): 0.049741
Total number of feature updates: 811928
Seconds required for this iteration: 0.227

***** Epoch #53 *****
Loss: 3195.084504
Improvement ratio: 0.108975
Feature L2-norm: 151.027360
Learning rate (eta): 0.049736
Total number of feature updates: 8

Märgendatud fail J2rva_Tyri_Kirna_id24064_1879a.json
Märgendatud fail Tartu_V6nnu_Ahja_id19682_1888a.json
Märgendatud fail Viljandi_Paistu_Holstre_id10573_1900a.json
Märgendatud fail Tartu_Kodavere_Ranna_id355_1883a.json
Märgendatud fail Harju_Hageri_Kohila_id4930_1887a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id23960_1881a.json
Märgendatud fail L22ne_Hanila_Massu_id361_1891a.json
Märgendatud fail Tartu_V6nnu_Kiidj2rve_id24846_1867a.json
Märgendatud fail Saare_P8ide_Laimjala_id8967_1915a.json
Märgendatud fail V6ru_Vastseliina_Misso_id20237_1882a.json
Märgendatud fail Harju_Hageri_Kohila_id11175_1875a.json
Märgendatud fail Harju_Hageri_Kohila_id23811_1873a.json
Märgendatud fail Tartu_V6nnu_Ahja_id19411_1887a.json
Märgendatud fail Tartu_Kodavere_Pala_id22605_1871a.json
Märgendatud fail Harju_Hageri_Kohila_id22513_1867a.json
Märgendatud fail Harju_Hageri_Kohila_id11216_1875a.json
Märgendatud fail Viljandi_K6pu_Suure-K6pu_id12875_1884a.json
Märgendatud fail Harju_Harju-Madise_Padise-

Märgendatud fail J2rva_Tyri_S2revere_id12373_1877a.json
Märgendatud fail Viljandi_Pilistvere_K6o_id24950_1843a.json
Märgendatud fail J2rva_Tyri_Kirna_id24928_1881a.json
Märgendatud fail P2rnu_Audru_V6lla_id5153_1877a.json
Märgendatud fail J2rva_Ambla_Uudekyla_id13881_1867a.json
Märgendatud fail Tartu_Kodavere_Alatskivi_id12243_1856a.json
Märgendatud fail J2rva_Tyri_S2revere_id13997_1886a.json
Märgendatud fail Tartu_V6nnu_Ahja_id21303_1889a.json
Märgendatud fail J2rva_Tyri_Kirna_id22874_1869a.json
Märgendatud fail V6ru_Vastseliina_Misso_id23231_1886a.json
Märgendatud fail Viljandi_Pilistvere_Arussaare_id23597_1845a.json
Märgendatud fail J2rva_Tyri_V22tsa_id16836_1886a.json
Märgendatud fail Tartu_V6nnu_Ahja_id14833_1882a.json
Märgendatud fail Harju_Juuru_Juuru_id556_1877a.json
Märgendatud fail Harju_Kose_Triigi_id9684_1869a.json
Märgendatud fail Tartu_R6ngu_Aakre_id6568_1826a.json
Märgendatud fail L22ne_Kullamaa_Kuij6e_id15492_1878a.json
Märgendatud fail Harju_Jyri_Rae_id5366_1890a.json
